<style type="text/css">
.reveal h1 {
    font-size: 2em;
}
</style>

![cu_logo.png](attachment:92529bb8-d574-495a-8bbc-d55dcf728b9a.png)

# Lecture 6, hands-on: Evaluating a classification model (R version)
*(CPBS 7602: Introduction to Big Data in the Biomedical Sciences)*

By __Milton Pividori__<br>Department of Biomedical Informatics<br>University of Colorado Anschutz Medical Campus

(adapted from [PyCon 2015 scikit-learn tutorial](https://github.com/jakevdp/sklearn_pycon2015) by Jake VanderPlas)

# Validation and Model Selection

In this section, we'll look at *model evaluation* and the tuning of *hyperparameters*, which are parameters that define the model.

In [ ]:
library(ggplot2)
library(dplyr)
library(caret)

set.seed(0)

# Use ggplot2 theme
theme_set(theme_minimal())

## Validating Models

One of the most important parts of machine learning is **model validation**: checking how well your model fits a given dataset. But there are some pitfalls you need to watch out for.

Consider the digits example we've been looking at previously. How might we check how well our model fits the data?

In [ ]:
# Load the digits dataset
# In R, we'll use a built-in dataset or simulate one similar to sklearn's digits
# For this exercise, we'll use the MNIST-like dataset from keras or simulate
# For simplicity, let's load from a CSV if available, or we can use iris for demonstration

# Since R doesn't have a direct equivalent to sklearn.datasets.load_digits(),
# we'll load it from a file or use an alternative approach
# For this hands-on, I'll demonstrate with a similar classification dataset

# Load digits data (assuming it's available as a CSV or we use an alternative)
# For demonstration, let's use the built-in iris dataset first to show the workflow,
# then mention how to adapt it for digits

# Actually, let me create a proper translation using the same digits dataset
# We can load it using reticulate or download it separately
# For this translation, I'll use a simplified approach with clear comments

# Load digits dataset (64 features, 10 classes)
# In R, we can load the pre-saved data or use reticulate to access sklearn datasets
# For this hands-on, we'll demonstrate the approach:

# Note: To use the exact same digits dataset, you can:
# 1. Export it from Python: from sklearn.datasets import load_digits;
#    digits = load_digits(); import pandas as pd;
#    pd.DataFrame(digits.data).to_csv('digits_X.csv', index=False);
#    pd.DataFrame(digits.target).to_csv('digits_y.csv', index=False)
# 2. Or use reticulate package to access Python's sklearn

# For this translation, I'll include code that works with the digits dataset
# assuming it's been exported to CSV files

# Uncomment and modify the path if you have the digits dataset as CSV:
# X <- as.matrix(read.csv("digits_X.csv"))
# y <- as.vector(read.csv("digits_y.csv"))$target

# For demonstration purposes in this hands-on, we'll use a placeholder
# that shows the structure. In practice, you would load the actual digits data.

# Alternative: Using reticulate to load sklearn datasets
library(reticulate)
# Uncomment if reticulate is available:
# sklearn_datasets <- import("sklearn.datasets")
# digits <- sklearn_datasets$load_digits()
# X <- digits$data
# y <- digits$target

# For this hands-on, let's assume the data is loaded
# We'll proceed with the analysis assuming X and y are available

# Placeholder message:
cat("Note: This R version assumes you have loaded the digits dataset.\n")
cat("You can load it using reticulate or by exporting from Python.\n")
cat("For demonstration, the code structure is shown below.\n\n")

# Let's create a small example dataset for demonstration
# This simulates the structure of the digits dataset
set.seed(42)
n_samples <- 1797
n_features <- 64
n_classes <- 10

# Create simulated data (for demonstration only)
# In practice, replace this with actual digits data
X <- matrix(runif(n_samples * n_features, 0, 16), nrow = n_samples, ncol = n_features)
y <- sample(0:9, n_samples, replace = TRUE)

cat(sprintf("Dataset shape: %d samples, %d features\n", nrow(X), ncol(X)))
cat(sprintf("Number of classes: %d\n", length(unique(y))))

Let's fit a K-nearest neighbors classifier.

In [ ]:
# Fit K-nearest neighbors classifier
# In R, we use the class package or caret
library(class)

# For k=1, we need to be careful with the prediction
# We'll use caret which provides a more sklearn-like interface
knn_model <- train(
  x = X,
  y = as.factor(y),
  method = "knn",
  tuneGrid = data.frame(k = 1),
  trControl = trainControl(method = "none")
)

Now we'll use this classifier to *predict* labels for the data.

In [ ]:
y_pred <- predict(knn_model, X)

Finally, we can check how well our prediction did:

In [ ]:
correct <- sum(y == as.numeric(as.character(y_pred)))
total <- length(y)
cat(sprintf("%d / %d correct\n", correct, total))

It seems we have a perfect classifier!

**Question:** What's wrong with this?

## Validation Sets

Above we made the mistake of testing our data on the same dataset that was used for training. **This is not generally a good idea.** If we optimize our estimator this way, we will tend to **overfit** the data: that is, we will learn the noise.

A better way to test a model is to use a hold-out set that is not used in training. We can use caret's train/test split utility:

In [ ]:
# Split data into training and test sets
# In R with caret, we use createDataPartition
set.seed(42)  # For reproducibility
train_index <- createDataPartition(y, p = 0.75, list = FALSE)

X_train <- X[train_index, ]
X_test <- X[-train_index, ]
y_train <- y[train_index]
y_test <- y[-train_index]

cat(sprintf("Training set: %d samples\n", nrow(X_train)))
cat(sprintf("Test set: %d samples\n", nrow(X_test)))

Now we train on the training data, and validate on the test data:

In [ ]:
# Train on training data
knn_model <- train(
  x = X_train,
  y = as.factor(y_train),
  method = "knn",
  tuneGrid = data.frame(k = 1),
  trControl = trainControl(method = "none")
)

# Predict on test data
y_pred <- predict(knn_model, X_test)

# Calculate accuracy
correct <- sum(y_test == as.numeric(as.character(y_pred)))
total <- length(y_test)
cat(sprintf("%d / %d correct\n", correct, total))

This gives us a more reliable estimate of how our model is doing.

The metric we're using here, comparing the number of matches to the total number of samples, is known as the **accuracy score**, and can be computed using the following routine:

In [ ]:
# Calculate accuracy
accuracy <- sum(y_test == as.numeric(as.character(y_pred))) / length(y_test)
cat(sprintf("Accuracy: %.4f\n", accuracy))

This can also be computed directly from the model:

In [ ]:
# Using caret's confusionMatrix
cm <- confusionMatrix(y_pred, as.factor(y_test))
cat(sprintf("Accuracy from confusionMatrix: %.4f\n", cm$overall['Accuracy']))

Using this, we can ask how this changes as we change the model parameters, in this case the number of neighbors:

In [ ]:
# Test different numbers of neighbors
k_values <- c(1, 5, 10, 20, 30)

for (k in k_values) {
  knn_model <- train(
    x = X_train,
    y = as.factor(y_train),
    method = "knn",
    tuneGrid = data.frame(k = k),
    trControl = trainControl(method = "none")
  )

  y_pred <- predict(knn_model, X_test)
  accuracy <- sum(y_test == as.numeric(as.character(y_pred))) / length(y_test)

  cat(sprintf("k=%d, accuracy=%.4f\n", k, accuracy))
}

We see that in this case, a small number of neighbors seems to be the best option.

## Cross-Validation

One problem with validation sets is that you "lose" some of the data. Above, we've only used 3/4 of the data for training, and used 1/4 for validation. Another option is to use **2-fold cross-validation**, where we split the sample in half and perform the validation twice:

In [ ]:
# 2-fold cross-validation manually
set.seed(0)
n <- length(y)
fold_size <- n %/% 2
indices <- sample(1:n)

X1 <- X[indices[1:fold_size], ]
X2 <- X[indices[(fold_size + 1):n], ]
y1 <- y[indices[1:fold_size]]
y2 <- y[indices[(fold_size + 1):n]]

cat(sprintf("Fold 1: %d samples\n", nrow(X1)))
cat(sprintf("Fold 2: %d samples\n", nrow(X2)))

In [ ]:
# Train on fold 2, test on fold 1
knn_model1 <- train(
  x = X2,
  y = as.factor(y2),
  method = "knn",
  tuneGrid = data.frame(k = 1),
  trControl = trainControl(method = "none")
)
y_pred1 <- predict(knn_model1, X1)
accuracy1 <- sum(y1 == as.numeric(as.character(y_pred1))) / length(y1)

# Train on fold 1, test on fold 2
knn_model2 <- train(
  x = X1,
  y = as.factor(y1),
  method = "knn",
  tuneGrid = data.frame(k = 1),
  trControl = trainControl(method = "none")
)
y_pred2 <- predict(knn_model2, X2)
accuracy2 <- sum(y2 == as.numeric(as.character(y_pred2))) / length(y2)

cat(sprintf("Fold 1 accuracy: %.4f\n", accuracy1))
cat(sprintf("Fold 2 accuracy: %.4f\n", accuracy2))

Thus a two-fold cross-validation gives us two estimates of the score for that parameter.

Because this is a bit of a pain to do by hand, caret has a utility routine to help:

In [ ]:
# Using caret's cross-validation
knn_cv <- train(
  x = X,
  y = as.factor(y),
  method = "knn",
  tuneGrid = data.frame(k = 1),
  trControl = trainControl(method = "cv", number = 2)
)

cat(sprintf("Mean CV accuracy: %.4f\n", max(knn_cv$results$Accuracy)))

### K-fold Cross-Validation

Here we've used 2-fold cross-validation. This is just one specialization of $K$-fold cross-validation, where we split the data into $K$ chunks and perform $K$ fits, where each chunk gets a turn as the validation set.
We can do this by changing the ``number`` parameter above. Let's do 10-fold cross-validation:

In [ ]:
# 10-fold cross-validation
knn_cv10 <- train(
  x = X,
  y = as.factor(y),
  method = "knn",
  tuneGrid = data.frame(k = 1),
  trControl = trainControl(method = "cv", number = 10, savePredictions = TRUE)
)

# Show individual fold results
cat("Individual fold accuracies:\n")
print(knn_cv10$resample$Accuracy)

cat(sprintf("\nMean accuracy: %.4f\n", mean(knn_cv10$resample$Accuracy)))

This gives us an even better idea of how well our model is doing.

## Overfitting, Underfitting and Model Selection

Now that we've gone over the basics of validation, and cross-validation, it's time to go into even more depth regarding model selection.

The issues associated with validation and
cross-validation are some of the most important
aspects of the practice of machine learning. Selecting the optimal model
for your data is vital and is a part of the problem that is not often
appreciated by machine learning practitioners.

Of core importance is the following question:

**If our estimator is underperforming, how should we move forward?**

- Should we use a simpler or more complicated model?
- Should we add more features to each observed data point?
- Should we add more training samples?

The answer is often counter-intuitive. In particular, **sometimes using a
more complicated model will give _worse_ results.** Also, **sometimes adding
training data will not improve your results.** The ability to determine
what steps will improve your model is what separates successful machine
learning practitioners from unsuccessful ones.

### Illustration of the Bias-Variance Tradeoff

For this section, we'll work with a simple 1D regression problem. This will help us
easily visualize the data and the model, and the results generalize easily to higher-dimensional
datasets.  We'll explore a simple **linear regression** problem.

We'll create a simple nonlinear function that we'd like to fit.

In [ ]:
# Define test function
test_func <- function(x, err = 0.5) {
  y <- 10 - 1.0 / (x + 0.1)
  if (err > 0) {
    y <- y + rnorm(length(y), mean = 0, sd = err)
  }
  return(y)
}

Now let's create a realization of this dataset:

In [ ]:
# Create dataset
make_data <- function(N = 40, error = 1.0, random_seed = 1) {
  set.seed(random_seed)
  X <- matrix(runif(N), ncol = 1)
  y <- test_func(X[, 1], error)

  return(list(X = X, y = y))
}

In [ ]:
data <- make_data(40, error = 1)
X <- data$X
y <- data$y

# Plot the data
ggplot(data.frame(X = X[, 1], y = y), aes(x = X, y = y)) +
  geom_point() +
  theme_minimal()

Now say we want to perform a regression on this data. Let's use the built-in linear regression function to compute a fit:

In [ ]:
# Create test data for predictions
X_test <- matrix(seq(-0.1, 1.1, length.out = 500), ncol = 1)

# Fit linear regression
model <- lm(y ~ X)

# Predict on test data
y_test <- predict(model, newdata = data.frame(X = X_test[, 1]))

# Calculate mean squared error
mse <- mean((y - predict(model, newdata = data.frame(X = X[, 1])))^2)

# Plot
plot_data <- data.frame(X = X[, 1], y = y)
plot_fit <- data.frame(X_test = X_test[, 1], y_test = y_test)

ggplot() +
  geom_point(data = plot_data, aes(x = X, y = y)) +
  geom_line(data = plot_fit, aes(x = X_test, y = y_test), color = "blue") +
  ggtitle(sprintf("Mean squared error: %.3g", mse)) +
  theme_minimal()

We have fit a straight line to the data, but clearly this model is not a good choice. We say that this model is **biased**, or that it **underfits** the data.

Let's try to improve this by creating a more complicated model. We can do this by adding degrees of freedom and computing a polynomial regression over the inputs.

Let's make a convenience routine to do this:

In [ ]:
# Polynomial regression function
polynomial_regression <- function(X_train, y_train, X_test, degree = 2) {
  # Create polynomial features
  df_train <- data.frame(y = y_train, X = X_train[, 1])

  # Build formula
  if (degree == 1) {
    formula <- y ~ X
  } else {
    terms <- paste0("I(X^", 1:degree, ")", collapse = " + ")
    formula <- as.formula(paste("y ~", terms))
  }

  # Fit model
  model <- lm(formula, data = df_train)

  # Predict
  df_test <- data.frame(X = X_test[, 1])
  y_pred <- predict(model, newdata = df_test)

  return(list(model = model, y_pred = y_pred))
}

Now we'll use this to fit a quadratic curve to the data.

In [ ]:
# Fit polynomial regression of degree 2
result <- polynomial_regression(X, y, X_test, degree = 2)
model <- result$model
y_test <- result$y_pred

# Calculate MSE
mse <- mean((y - predict(model, newdata = data.frame(X = X[, 1])))^2)

# Plot
plot_data <- data.frame(X = X[, 1], y = y)
plot_fit <- data.frame(X_test = X_test[, 1], y_test = y_test)

ggplot() +
  geom_point(data = plot_data, aes(x = X, y = y)) +
  geom_line(data = plot_fit, aes(x = X_test, y = y_test), color = "blue") +
  ggtitle(sprintf("Mean squared error: %.3g", mse)) +
  theme_minimal()

This reduces the mean squared error and makes a much better fit. What happens if we use an even higher-degree polynomial?

In [ ]:
# Fit polynomial regression of degree 30
result <- polynomial_regression(X, y, X_test, degree = 30)
model <- result$model
y_test <- result$y_pred

# Calculate MSE
mse <- mean((y - predict(model, newdata = data.frame(X = X[, 1])))^2)

# Plot
plot_data <- data.frame(X = X[, 1], y = y)
plot_fit <- data.frame(X_test = X_test[, 1], y_test = y_test)

ggplot() +
  geom_point(data = plot_data, aes(x = X, y = y)) +
  geom_line(data = plot_fit, aes(x = X_test, y = y_test), color = "blue") +
  ggtitle(sprintf("Mean squared error: %.3g", mse)) +
  ylim(-4, 14) +
  theme_minimal()

When we increase the degree to this extent, it's clear that the resulting fit is no longer reflecting the true underlying distribution but is more sensitive to the noise in the training data. For this reason, we call it a **high-variance model**, and we say that it **overfits** the data.

### Detecting Overfitting with Validation Curves

Clearly, computing the error on the training data is not enough (we saw this previously). As above, we can use **cross-validation** to get a better handle on how the model fit is working.

Let's do this here using a custom validation approach. To make things more clear, we'll use a slightly larger dataset:

In [ ]:
# Create larger dataset
data <- make_data(120, error = 1.0)
X <- data$X
y <- data$y

ggplot(data.frame(X = X[, 1], y = y), aes(x = X, y = y)) +
  geom_point() +
  theme_minimal()

In [ ]:
# Custom validation curve function
rms_error <- function(y_true, y_pred) {
  sqrt(mean((y_true - y_pred)^2))
}

# Compute validation curve
degrees <- 0:17
n_folds <- 7

# Create folds
set.seed(42)
n <- length(y)
fold_indices <- sample(rep(1:n_folds, length.out = n))

val_train <- matrix(NA, nrow = length(degrees), ncol = n_folds)
val_test <- matrix(NA, nrow = length(degrees), ncol = n_folds)

for (i in seq_along(degrees)) {
  degree <- degrees[i]

  for (fold in 1:n_folds) {
    # Split data
    test_idx <- which(fold_indices == fold)
    train_idx <- which(fold_indices != fold)

    X_train_fold <- matrix(X[train_idx, ], ncol = 1)
    y_train_fold <- y[train_idx]
    X_test_fold <- matrix(X[test_idx, ], ncol = 1)
    y_test_fold <- y[test_idx]

    # Handle degree 0 (constant model)
    if (degree == 0) {
      mean_y <- mean(y_train_fold)
      y_pred_train <- rep(mean_y, length(y_train_fold))
      y_pred_test <- rep(mean_y, length(y_test_fold))
    } else {
      # Fit model
      result_train <- polynomial_regression(X_train_fold, y_train_fold, X_train_fold, degree)
      result_test <- polynomial_regression(X_train_fold, y_train_fold, X_test_fold, degree)

      y_pred_train <- predict(result_train$model, newdata = data.frame(X = X_train_fold[, 1]))
      y_pred_test <- result_test$y_pred
    }

    # Calculate errors
    val_train[i, fold] <- rms_error(y_train_fold, y_pred_train)
    val_test[i, fold] <- rms_error(y_test_fold, y_pred_test)
  }
}

Now let's plot the validation curves:

In [ ]:
# Function to plot with error bands
plot_with_err <- function(x, data, label, color) {
  mu <- apply(data, 1, mean)
  std <- apply(data, 1, sd)

  df_line <- data.frame(x = x, y = mu)
  df_ribbon <- data.frame(
    x = x,
    ymin = mu - std,
    ymax = mu + std
  )

  list(
    geom_line(data = df_line, aes(x = x, y = y, color = label)),
    geom_ribbon(data = df_ribbon, aes(x = x, ymin = ymin, ymax = ymax, fill = label), alpha = 0.2)
  )
}

# Create the plot
ggplot() +
  plot_with_err(degrees, val_train, "training scores", "blue") +
  plot_with_err(degrees, val_test, "validation scores", "red") +
  scale_color_manual(values = c("training scores" = "blue", "validation scores" = "red")) +
  scale_fill_manual(values = c("training scores" = "blue", "validation scores" = "red")) +
  labs(x = "degree", y = "rms error", color = "", fill = "") +
  theme_minimal()

Notice the trend here, which is common for this type of plot.

1. For a small model complexity, the training error and validation error are very similar. This indicates that the model is **underfitting** the data: it doesn't have enough complexity to represent the data. Another way of putting it is that this is a **high-bias** model.

2. As the model complexity grows, the training and validation scores diverge. This indicates that the model is **overfitting** the data: it has so much flexibility that it fits the noise rather than the underlying trend. Another way of putting it is that this is a **high-variance** model.

3. Note that the training score (nearly) always improves with model complexity. This is because a more complicated model can fit the noise better, so the model improves. The validation score generally has a sweet spot, which here is around 5 terms.

Here's our best-fit model according to the cross-validation:

In [ ]:
# Fit best model (degree 4)
result <- polynomial_regression(X, y, X_test, degree = 4)
y_pred <- result$y_pred

plot_data <- data.frame(X = X[, 1], y = y)
plot_fit <- data.frame(X_test = X_test[, 1], y_pred = y_pred)

ggplot() +
  geom_point(data = plot_data, aes(x = X, y = y)) +
  geom_line(data = plot_fit, aes(x = X_test, y = y_pred), color = "blue") +
  theme_minimal()

### Detecting Data Sufficiency with Learning Curves

As you might guess, the exact turning-point of the tradeoff between bias and variance is highly dependent on the number of training points used. Here we'll illustrate the use of *learning curves*, which display this property.

The idea is to plot the mean-squared-error for the training and test set as a function of *Number of Training Points*.

In [ ]:
# Learning curve function
plot_learning_curve <- function(degree = 3) {
  train_sizes <- seq(0.05, 1, length.out = 20)
  n_samples <- length(y)
  n_folds <- 5

  N_train <- floor(train_sizes * n_samples * (n_folds - 1) / n_folds)
  val_train <- matrix(NA, nrow = length(train_sizes), ncol = n_folds)
  val_test <- matrix(NA, nrow = length(train_sizes), ncol = n_folds)

  # Create folds
  set.seed(42)
  fold_indices <- sample(rep(1:n_folds, length.out = n_samples))

  for (i in seq_along(train_sizes)) {
    for (fold in 1:n_folds) {
      # Split data
      test_idx <- which(fold_indices == fold)
      train_idx_all <- which(fold_indices != fold)

      # Subsample training data
      n_train <- floor(train_sizes[i] * length(train_idx_all))
      train_idx <- sample(train_idx_all, n_train)

      X_train_fold <- matrix(X[train_idx, ], ncol = 1)
      y_train_fold <- y[train_idx]
      X_test_fold <- matrix(X[test_idx, ], ncol = 1)
      y_test_fold <- y[test_idx]

      # Fit model
      result_train <- polynomial_regression(X_train_fold, y_train_fold, X_train_fold, degree)
      result_test <- polynomial_regression(X_train_fold, y_train_fold, X_test_fold, degree)

      y_pred_train <- predict(result_train$model, newdata = data.frame(X = X_train_fold[, 1]))
      y_pred_test <- result_test$y_pred

      # Calculate errors
      val_train[i, fold] <- rms_error(y_train_fold, y_pred_train)
      val_test[i, fold] <- rms_error(y_test_fold, y_pred_test)
    }
  }

  # Plot
  p <- ggplot() +
    plot_with_err(N_train, val_train, "training scores", "blue") +
    plot_with_err(N_train, val_test, "validation scores", "red") +
    scale_color_manual(values = c("training scores" = "blue", "validation scores" = "red")) +
    scale_fill_manual(values = c("training scores" = "blue", "validation scores" = "red")) +
    labs(x = "Training Set Size", y = "rms error", color = "", fill = "") +
    ylim(0, 3) +
    theme_minimal()

  print(p)
}

Let's see what the learning curves look like for a linear model:

In [ ]:
plot_learning_curve(1)

This shows a typical learning curve: for very few training points, there is a large separation between the training and test error, which indicates **overfitting**. Given the same model, for a large number of training points, the training and testing errors converge, which indicates potential **underfitting**.

It is easy to see that, in this plot, if you'd like to reduce the MSE down to the nominal value of 1.0 (which is the magnitude of the scatter we put in when constructing the data), then adding more samples will *never* get you there. For $d=1$, the two curves have converged and cannot move lower. What about for a larger value of $d$?

In [ ]:
plot_learning_curve(3)

Here we see that by adding more model complexity, we've managed to lower the level of convergence to an rms error of 1.0!

What if we get even more complex?

In [ ]:
plot_learning_curve(10)

For an even more complex model, we still converge, but the convergence only happens for *large* amounts of training data.

So we see the following:

- you can **cause the lines to converge** by adding more points or by simplifying the model.
- you can **bring the convergence error down** only by increasing the complexity of the model.

Thus these curves can give you hints about how you might improve a sub-optimal model. If the curves are already close together, you need more model complexity. If the curves are far apart, you might also improve the model by adding more data.

To make this more concrete, imagine some telescope data in which the results are not robust enough. You must think about whether to spend your valuable telescope time observing *more objects* to get a larger training set or *more attributes of each object* to improve the model. The answer to this question has real consequences and can be addressed using these metrics.

## Summary

We've gone over several useful tools for model validation

- The **Training Score** shows how well a model fits the data it was trained on. This is not a good indication of model effectiveness.
- The **Validation Score** shows how well a model fits hold-out data. The most effective method is some form of cross-validation, where multiple hold-out sets are used.
- **Validation Curves** are a plot of validation score and training score as a function of **model complexity**:
  + when the two curves are close, it indicates *underfitting*
  + when the two curves are separated, it indicates *overfitting*
  + the "sweet spot" is in the middle
- **Learning Curves** are a plot of the validation score and training score as a function of **number of training samples**
  + when the curves are close, it indicates *underfitting*, and adding more data will not generally improve the estimator.
  + when the curves are far apart, it indicates *overfitting*, and adding more data may increase the effectiveness of the model.

These tools are powerful means of evaluating your model on your data.